In [1]:
import curl

import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

curl.init()

/Users/mathiasleys/projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using Communicator type:  <class 'curl.communicator.distributed_communicator.DistributedCommunicator'>
[<>] Waiting for connections...
[<>] DEFAULT ARGS: {'DISTRIBUTED_BACKEND': 'gloo', 'RENDEZVOUS': 'file:///tmp/vcrypten-Bcrypten-Zcrypten-gcrypten-Ecrypten-Gcrypten-xcrypten-gcrypten-mcrypten-l', 'WORLD_SIZE': 1, 'RANK': 0, 'TTP': False}
[Device] LUTs initialized for cpu



In [2]:
MODEL_NAME = "iiiorg/piiranha-v1-detect-personal-information"

model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [3]:
# NOTE: think could also have a different name like `n_embd` in the config, depending on the model
hidden_size = model.config.hidden_size

In [4]:
# To ensure a constant input size, we will always pad or truncate to a fixed size
INPUT_TEXT = "this is a sentence"

encoded_input = tokenizer(
    INPUT_TEXT,
    return_tensors='pt',
    padding="max_length",
    truncation=True,
    max_length=hidden_size,
)

output = model.forward(
    input_ids=encoded_input["input_ids"],
    attention_mask=encoded_input["attention_mask"],
)

In [5]:
# shape: batch_size x num tokens x num classes
# interpretation: probability logit for every class per token per sequence in the batch
output.logits.shape

torch.Size([1, 768, 18])

In [6]:
predicted_classes = torch.argmax(output.logits, dim=-1)
print("Predictions are: ", predicted_classes)

Predictions are:  tensor([[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
         1

In [7]:
private_model = curl.nn.from_pytorch(model, encoded_input["input_ids"])

/Users/mathiasleys/projects/.venv/lib/python3.12/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(mid - 1).type_as(relative_pos),
/Users/mathiasleys/projects/.venv/lib/python3.12/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:551: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.ceil(torch.log(abs_pos / mid) / torch.log(torch.tensor((max_position - 1) / mid)) * (mid - 1)) 

In [8]:
private_model.encrypt()

Graph encrypted module

In [9]:
private_input = curl.cryptensor(encoded_input["input_ids"], precision=0)

In [10]:
private_output = private_model.forward(private_input)

/Users/mathiasleys/projects/aivm-curl/curl/__init__.py:472: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:305.)
  condition = condition.float()


In [11]:
predicted_classes = torch.argmax(private_output.get_plain_text(), dim=-1)
print("Predictions are: ", predicted_classes)

Predictions are:  tensor([[ 1,  4, 11, 15,  4,  9,  6, 15,  9,  7,  8, 14,  4,  4, 12, 17,  1,  0,
          3, 17, 17,  9,  9,  1, 10,  6,  1, 16,  8,  0,  3, 15,  4,  1, 14, 11,
          0, 17,  9,  3, 17, 10, 13,  9, 17, 10,  8, 11, 10,  1,  6, 11,  0,  4,
         11,  5,  1, 14,  2, 10, 14, 17,  7,  0, 16, 12,  8,  7,  4,  1, 15,  9,
          7,  5,  3, 14,  9, 14, 15, 15,  9, 13, 11,  5,  5,  6, 15,  6,  9, 15,
         11,  3,  1, 16,  1, 15,  9,  1, 15, 14, 10, 16,  9,  3,  1, 15,  1, 10,
         16,  4,  4, 17, 12,  3, 12, 10, 14, 14,  1, 16, 10,  2,  3, 14,  4,  7,
          7,  4, 10,  1, 13,  1,  6,  0,  9,  6,  3, 14,  1, 14, 10, 16,  1, 17,
          3,  7, 15, 14, 12,  7,  1, 13, 14, 10, 14, 12, 15,  3,  1,  2,  5, 15,
         14, 15,  6,  6, 11,  0,  1,  5,  9, 10, 13, 15, 15,  1,  5, 15,  4,  4,
         15,  7,  9, 15,  1,  8,  6,  4,  1,  1,  9,  9, 15, 15, 15,  5,  5,  4,
         12, 17, 12, 13, 10,  6,  6,  0,  2,  0,  9,  4, 15, 12,  5,  5, 12,  7,
         1